#### Overview
* GeoCoding: 주소정보를 좌표정보로 치환하는 방법
* 구글, 네이버, 카카오 등에서 변경하는 API를 제공하며 여기서는 `네이버API` 활용

#### Import Libraries

In [25]:
import os
import pandas as pd
import geopandas as gpd
import folium

from shapely.geometry import Point

import geo_coding_31 as gc

#### Simple Test

In [5]:
address_name = '전남 나주시 중야1길 15'
geo_coding = gc.get_geocoding(address_name)
geo_coding

{'status': 'OK',
 'meta': {'totalCount': 1, 'page': 1, 'count': 1},
 'addresses': [{'roadAddress': '전라남도 나주시 중야1길 15 대방엘리움 2차 아파트',
   'jibunAddress': '전라남도 나주시 빛가람동 374 대방엘리움 2차 아파트',
   'englishAddress': '15, Jungya 1-gil, Naju-si, Jeollanam-do, Republic of Korea',
   'addressElements': [{'types': ['SIDO'],
     'longName': '전라남도',
     'shortName': '전라남도',
     'code': ''},
    {'types': ['SIGUGUN'], 'longName': '나주시', 'shortName': '나주시', 'code': ''},
    {'types': ['DONGMYUN'],
     'longName': '빛가람동',
     'shortName': '빛가람동',
     'code': ''},
    {'types': ['RI'], 'longName': '', 'shortName': '', 'code': ''},
    {'types': ['ROAD_NAME'],
     'longName': '중야1길',
     'shortName': '중야1길',
     'code': ''},
    {'types': ['BUILDING_NUMBER'],
     'longName': '15',
     'shortName': '15',
     'code': ''},
    {'types': ['BUILDING_NAME'],
     'longName': '대방엘리움 2차 아파트',
     'shortName': '대방엘리움 2차 아파트',
     'code': ''},
    {'types': ['LAND_NUMBER'],
     'longName': '374',
     

In [6]:
geo_coding['addresses'][0]['x'], geo_coding['addresses'][0]['y']

('126.7789937', '35.0175096')

##### Display Explore

* with folium library

In [26]:
lat, lon = geo_coding['addresses'][0]['y'], geo_coding['addresses'][0]['x']

In [31]:
base_map = folium.Map(location=[lat,lon], tiles='cartodbpositron', zoom_start=17)
folium.Marker([lat, lon], popup='My Home').add_to(base_map)
base_map

* with explore()

In [20]:
my_home_gdf = gpd.GeoDataFrame(
    {'position': ['My Home']},
    geometry=[Point(x,y) for x, y in zip([geo_coding['addresses'][0]['x']], [geo_coding['addresses'][0]['y']])],
    crs='epsg:4326'
)

In [21]:
my_home_gdf

,position,geometry
0,My Home,POINT (126.77899 35.01751)


In [22]:
my_home_gdf.explore(
    tiles='http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}&s=Ga', 
    attr='google',
)

#### Fetching Data

In [7]:
CCTV_CSV_PATH = os.path.join(
    os.getcwd(), '89_data', 'csv', 'road_cctv_info.csv'
)
cctv_df = pd.read_csv(
    CCTV_CSV_PATH, encoding='EUC-KR', header=0, index_col=None,
    names=[
        'admin_nm', 'addr_rd', 'addr_jb', 'purpose', 'cctv_cnt', 
        'pixel', 'direction', 'period', 'date_install', 'tel', 
        'lat', 'lon', 'date_make', 'offer_admin_cd', 'offer_admin_nm'
    ],
    dtype={
        'cctv_cnt':object, 'pixel':object, 'period':object, 
        'lat':float, 'lon':float, 'offer_admin_cd':object
    }, 
    thousands=',', # 천단위 쉼표 제거(?)
    nrows=100, # 100건
)

In [8]:
cctv_df.shape

(100, 15)

In [9]:
cctv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   admin_nm        100 non-null    object 
 1   addr_rd         29 non-null     object 
 2   addr_jb         100 non-null    object 
 3   purpose         100 non-null    object 
 4   cctv_cnt        100 non-null    object 
 5   pixel           86 non-null     object 
 6   direction       100 non-null    object 
 7   period          100 non-null    object 
 8   date_install    98 non-null     object 
 9   tel             100 non-null    object 
 10  lat             100 non-null    float64
 11  lon             100 non-null    float64
 12  date_make       100 non-null    object 
 13  offer_admin_cd  100 non-null    object 
 14  offer_admin_nm  100 non-null    object 
dtypes: float64(2), object(13)
memory usage: 11.8+ KB


In [10]:
cctv_df.head()

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm
0,경상남도 김해시청,NaN,경상남도 김해시 구산동 1072-19,생활방범,3,200,현대병원뒤,30,2019-11,055-330-4741,35.249247,128.871639,2019-06-26,5350000,경상남도 김해시
1,경상남도 김해시청,NaN,경상남도 김해시 구산동 305-15,생활방범,2,200,목화골공원,30,2019-12,055-330-4741,35.247411,128.873605,2019-06-26,5350000,경상남도 김해시
2,경상남도 김해시청,NaN,경상남도 김해시 삼계동 1484-14,생활방범,2,200,정원빌라,30,2019-06,055-330-4741,35.261728,128.874144,2019-06-26,5350000,경상남도 김해시
3,경상남도 김해시청,NaN,경상남도 김해시 구산동 1043,생활방범,4,200,최가아구찜,30,2019-05,055-330-4741,35.247076,128.872461,2019-06-26,5350000,경상남도 김해시
4,경상남도 김해시청,NaN,경상남도 김해시 구산동 175-10,생활방범,2,200,동호맨션,30,2019-12,055-330-4741,35.241185,128.877334,2019-06-26,5350000,경상남도 김해시


In [11]:
cctv_df.isnull().sum()

admin_nm           0
addr_rd           71
addr_jb            0
purpose            0
cctv_cnt           0
pixel             14
direction          0
period             0
date_install       2
tel                0
lat                0
lon                0
date_make          0
offer_admin_cd     0
offer_admin_nm     0
dtype: int64

In [12]:
cctv_df.dropna(subset=['addr_jb'], axis=0, inplace=True)
cctv_df.shape
# 삭제된 row는 없음

(100, 15)

#### GeoCoding in DataFrame

In [32]:
gc_lon = []
gc_lat = []
length_df = cctv_df.shape[0]
print(length_df)

100


In [33]:
for idx in range(length_df):
    addr = cctv_df.iloc[idx]['addr_jb']
    _gc_data = gc.get_geocoding(addr)
    try:
        _gc_geometry = _gc_data['addresses'][0]
        gc_lon.append(_gc_geometry['x'])
        gc_lat.append(_gc_geometry['y'])
    except:
        gc_lon.append(0)
        gc_lat.append(0)    

In [36]:
cctv_df['lon2'] = gc_lon
cctv_df['lat2'] = gc_lat
cctv_df.head()

,admin_nm,addr_rd,addr_jb,purpose,cctv_cnt,pixel,direction,period,date_install,tel,lat,lon,date_make,offer_admin_cd,offer_admin_nm,lon2,lat2
0,경상남도 김해시청,NaN,경상남도 김해시 구산동 1072-19,생활방범,3,200,현대병원뒤,30,2019-11,055-330-4741,35.249247,128.871639,2019-06-26,5350000,경상남도 김해시,128.8716256,35.2496502
1,경상남도 김해시청,NaN,경상남도 김해시 구산동 305-15,생활방범,2,200,목화골공원,30,2019-12,055-330-4741,35.247411,128.873605,2019-06-26,5350000,경상남도 김해시,128.8734019,35.2473902
2,경상남도 김해시청,NaN,경상남도 김해시 삼계동 1484-14,생활방범,2,200,정원빌라,30,2019-06,055-330-4741,35.261728,128.874144,2019-06-26,5350000,경상남도 김해시,128.8741368,35.2618309
3,경상남도 김해시청,NaN,경상남도 김해시 구산동 1043,생활방범,4,200,최가아구찜,30,2019-05,055-330-4741,35.247076,128.872461,2019-06-26,5350000,경상남도 김해시,128.8718889,35.2464445
4,경상남도 김해시청,NaN,경상남도 김해시 구산동 175-10,생활방범,2,200,동호맨션,30,2019-12,055-330-4741,35.241185,128.877334,2019-06-26,5350000,경상남도 김해시,128.8773246,35.2411248


##### Display

In [37]:
cctv_gdf = gpd.GeoDataFrame(cctv_df, geometry=gpd.points_from_xy(cctv_df.lon2, cctv_df.lat2), crs=4326)

In [42]:
cctv_gdf.geometry.x.min(), cctv_gdf.geometry.y.min()

(0.0, 0.0)

In [44]:
cctv_gdf = cctv_gdf[cctv_gdf.geometry.x != 0.0]

In [45]:
cctv_gdf.explore(color='red', tiles='http://basemaps.cartocdn.com/dark_all/{z}/{x}/{y}.png', attr='Carto')

In [35]:
base_map = folium.Map(location=[gc_lat[0],gc_lon[0]], tiles='cartodbpositron', zoom_start=14)
for idx in range(len(gc_lat)):
    folium.Marker([gc_lat[idx], gc_lon[idx]], popup=cctv_df.iloc[idx]['direction']).add_to(base_map)
base_map